In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13577,2024-03-28,Eua Nba,21:00,New Orleans Pelicans,Milwaukee Bucks,2.08,1.80,222.5,1.81,1.95,1.5,1.93,1.64,Io2mliW1,0.480769,0.555556,0.552486,0.512821,0.036325,0.6,0.4,NaN,NaN,192.140,23.744428,0.123579,201.034,42.848757,0.213142,179.514,199.098,228.48,176.72,0.0,0.0,0.0,0.0,0.102057,0.052657,0.114880,-1.19,-0.238,-4.537815,0.670456,0.5,-0.170456,-1.53,-0.306,-2.614379,0.565212,0.4,-0.165212
13578,2024-03-28,Europa Euroliga,16:30,Baskonia,Anadolu Efes,1.67,2.28,170.5,1.91,1.91,-3.5,1.95,2.90,lQSypWLg,0.598802,0.438596,0.523560,0.523560,0.037399,0.4,0.6,NaN,NaN,184.900,90.574146,0.489855,147.690,28.397642,0.192279,131.560,271.364,141.68,116.00,0.0,0.0,0.0,0.0,0.218398,0.000000,0.277011,1.42,0.284,2.359155,0.600262,0.7,0.099738,-5.00,-1.000,-1.280000,0.434345,0.1,-0.334345
13579,2024-03-28,Europa Euroliga,16:30,Bayern,Barcelona,2.64,1.51,156.5,1.87,1.95,3.5,2.00,1.29,zJn1jYrQ,0.378788,0.662252,0.534759,0.512821,0.041040,0.2,0.4,NaN,NaN,148.360,49.799775,0.335668,152.404,49.462257,0.324547,137.064,166.728,133.20,212.22,0.0,0.0,0.0,0.0,0.385075,0.029617,0.305195,-0.06,-0.012,-136.666667,0.581575,0.5,-0.081575,-1.47,-0.294,-1.734694,0.541895,0.3,-0.241895
13580,2024-03-28,Europa Euroliga,16:30,Partizan,Olympiakos,1.80,2.05,158.5,1.91,1.91,-2.5,2.00,2.40,hbZSohis,0.555556,0.487805,0.523560,0.523560,0.043360,0.2,0.4,NaN,NaN,124.128,45.194204,0.364094,136.146,21.341766,0.156756,109.644,234.910,155.80,129.60,0.0,0.0,0.0,0.0,0.091832,0.000000,0.128565,-2.67,-0.534,-1.498127,0.832267,0.7,-0.132267,-1.29,-0.258,-4.069767,0.508383,0.4,-0.108383
13581,2024-03-28,Austrália Nbl,05:30,Tasmania JackJumpers,Melbourne United,1.93,1.88,179.5,1.87,1.79,-1.5,2.03,1.98,hMHGN0Gq,0.518135,0.531915,0.534759,0.558659,0.050050,0.6,0.4,NaN,NaN,142.964,31.691846,0.221677,141.592,24.883993,0.175744,161.832,144.944,148.42,154.00,0.0,0.0,0.0,0.0,0.018559,0.030912,0.017634,1.70,0.340,2.735294,0.661368,0.6,-0.061368,-3.54,-0.708,-1.242938,0.631669,0.3,-0.331669
13582,2024-03-28,China Cba,08:35,Fujian,Shandong,2.64,1.46,219.5,1.86,1.88,1.5,2.03,1.52,W2RDObN6,0.378788,0.684932,0.537634,0.531915,0.063719,0.6,0.4,NaN,NaN,294.668,210.159373,0.713207,277.984,149.353580,0.537274,198.514,265.842,118.56,212.16,0.0,0.0,0.0,0.0,0.407018,0.007563,0.203169,0.96,0.192,8.541667,0.441784,0.5,0.058216,-5.00,-1.000,-0.460000,0.408236,0.2,-0.208236
13583,2024-03-28,Coreia Do Sul Kbl,07:00,Seoul Knights,LG Sakers,1.95,1.83,157.5,1.85,1.87,1.5,1.80,1.69,bRKHzwl5,0.512821,0.546448,0.540541,0.534759,0.059269,0.4,0.4,NaN,NaN,146.524,29.597799,0.202000,167.004,14.143121,0.084687,130.716,158.430,137.60,160.59,0.0,0.0,0.0,0.0,0.044896,0.007603,0.044574,-1.94,-0.388,-2.448454,0.625965,0.4,-0.225965,2.90,0.580,1.431034,0.502643,0.8,0.297357
13584,2024-03-28,Coreia Do Sul Wkbl Feminina,07:00,Woori WON F,KB Stars F,2.09,1.72,128.5,1.89,1.89,1.5,1.90,1.56,tzUtGzKh,0.478469,0.581395,0.529101,0.529101,0.059864,0.6,0.8,NaN,NaN,113.872,40.101336,0.352162,88.354,12.780981,0.144657,111.826,95.326,181.60,91.06,0.0,0.0,0.0,0.0,0.137338,0.000000,0.138969,-0.34,-0.068,-16.029412,0.711554,0.8,0.088446,0.17,0.034,21.176471,0.809325,0.7,-0.109325
13585,2024-03-28,Eua Ncaa,23:09,Iowa State,Illinois,1.80,2.05,145.5,1.80,1.95,-2.5,2.00,2.35,EX1iYLLc,0.555556,0.487805,0.555556,0.512821,0.043360,0.6,0.8,NaN,NaN,166.666,95.091562,0.570552,124.982,24.202070,0.193644,99.296,195.806,91.12,133.59,0.0,0.0,0.0,0.0,0.091832,0.056569,0.113787,1.6

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,16:30,Europa Euroliga,Baskonia,Anadolu Efes,170.5,1.91,1.0,Over
1,16:30,Europa Euroliga,Partizan,Olympiakos,158.5,1.91,1.0,Over
2,05:30,Austrália Nbl,Tasmania JackJumpers,Melbourne United,179.5,1.87,1.0,Over
3,23:09,Eua Ncaa,Iowa State,Illinois,145.5,1.80,1.0,Over
4,12:40,Israel Wbl Feminina,Maccabi Ashdod F,Hapoel Jerusalem F,156.5,1.86,1.0,Over
5,07:00,Romênia Liga Nacional Feminina,Rapid Bucuresti F,Olimpia Brasov F,145.5,1.85,1.0,Over
6,14:00,Rússia Liga Vtb United,CSKA Moscow,Zenit Petersburg,158.5,1.86,1.0,Over
7,12:00,Rússia Liga Vtb United,Samara,Uralmash Ekaterinburg,156.5,1.81,1.0,Over
8,14:00,Rússia Liga Vtb United,Runa Basket Moscow,Nizhny Novgorod,157.5,1.86,1.0,Over
9,15:00,Europa Euroliga,Zalgiris Kaunas,Milano,158.5,1.91,1.0,Over
